In [ ]:
from pyscf import gto, scf, dft, fci, mcscf
import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=200)

### Test functionals

In cp2k we do not have the LDAERF functional, so we need to construct it with a weird combination of other functionals. Because we do this with libxc, and pyscf also has libxc, we can test directly in pyscf that the combination of functionals used by cp2k does actually corresponds to LDAERF (that in pyscf is available with the xcfun library).

### XCFUN

In [ ]:
omega = 0.5

mol = gto.M(
    atom = 'C  0.0      0.0      0.66242; \
            C  0.0      0.0     -0.66242; \
            H -0.12018  0.91284  1.23164; \
            H  0.12018 -0.91284  1.23164; \
            H  0.12018  0.91284 -1.23164; \
            H -0.12018 -0.91284 -1.23164;',
    basis="sto3g",
    verbose=4,
)

mf_xcfun = dft.RKS(mol)
mf_xcfun.chkfile = None
mf_xcfun._numint.libxc = dft.xcfun
mf_xcfun.xc = 'LDAERF + LR_HF(0.5)'

NameError: name 'gto' is not defined

In [ ]:
mf_xcfun.kernel()

NameError: name 'mf_xcfun' is not defined

### LibXC

Unfortunately this does not work. I don't know why, but apparently LDA_ERF is not supported in pyscf through libxc.

In [ ]:
mf_libxc = dft.rks.RKS(mol)
# this is how we define the functional in cp2k
mf_libxc.xc = 'LDA_X_ERF + LR_HF(0.5) - 1.0*LDA_C_PMGB06 + LDA_C_PW'
mf_libxc.chkfile = None

Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7ffb3b8956c0>
Traceback (most recent call last):
  File "/home/stefano/Miniconda/envs/qiskit/lib/python3.10/tempfile.py", line 450, in __del__
    self.close()
  File "/home/stefano/Miniconda/envs/qiskit/lib/python3.10/tempfile.py", line 446, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/home/stefano/Projects/embedding/periodic-embedding-data/test/tmpw2d8ne39'


In [ ]:
mf_libxc.kernel()